Notebook updated 7/17/22 -- initial data analysis to aggregate RT data to hourly totals

*This notebook will not work without access to the private S3 bucket; work is planned to split some of this notebook out to automated scripts to create the daily CSV files

In [1]:
import boto3
import json
import pandas as pd
import pendulum

In [2]:
start_date = '2022-07-17'
end_date = '2022-08-07'

date_range = [d for d in pendulum.period(pendulum.from_format(start_date, 'YYYY-MM-DD'), pendulum.from_format(end_date, 'YYYY-MM-DD')).range('days')]

In [3]:
# get objects from S3
# this requires being locally authenticated: https://docs.aws.amazon.com/cli/latest/userguide/cli-configure-files.html

s3 = boto3.resource('s3')
bucket = s3.Bucket('chn-ghost-buses-private')

In [4]:
for day in date_range:
    date_str = day.to_date_string()
    print(f"Processing {date_str} at {pendulum.now().to_datetime_string()}")
    objects = bucket.objects.filter(Prefix = f'bus_data/{date_str}')
    
    print(f"------- loading data at {pendulum.now().to_datetime_string()}")
    
    # load data
    data_dict = {}

    for obj in objects:
        # print(f"loading {obj}")
        obj_name = obj.key
        # https://stackoverflow.com/questions/31976273/open-s3-object-as-a-string-with-boto3
        obj_body = json.loads(obj.get()['Body'].read().decode('utf-8'))
        data_dict[obj_name] = obj_body
    
    # parse data into actual vehicle locations and errors
    
    print(f"------- parsing data at {pendulum.now().to_datetime_string()}")

    data = pd.DataFrame()
    errors = pd.DataFrame()

    # k, v here are filename: full dict of JSON
    for k, v in data_dict.items():
        # print(f"processing {k}")
        filename = k
        new_data = pd.DataFrame()
        new_errors = pd.DataFrame()
        # expect ~12 "chunks" per JSON
        for chunk, contents in v.items():
            if 'vehicle' in v[chunk]['bustime-response'].keys():
                new_data = new_data.append(pd.DataFrame(v[chunk]['bustime-response']['vehicle']))
            if 'error' in v[chunk]['bustime-response'].keys():
                new_errors = new_errors.append(pd.DataFrame(v[chunk]['bustime-response']['error']))
        new_data['scrape_file'] = filename
        new_errors['scrape_file'] = filename
        data = data.append(new_data)
        errors = errors.append(new_errors)
        
    print(f"------- saving data at {pendulum.now().to_datetime_string()}")

    if len(errors) > 0:
        bucket.put_object(Body = errors.to_csv(index = False), 
                   Key = f'bus_full_day_errors_v2/{date_str}.csv')

    if len(data) > 0:
        # convert data time to actual datetime
        data['data_time'] = pd.to_datetime(data['tmstmp'], format='%Y%m%d %H:%M')

        data['data_hour'] = data.data_time.dt.hour
        data['data_date'] = data.data_time.dt.date

        bucket.put_object(Body = data.to_csv(index = False), 
                   Key = f'bus_full_day_data_v2/{date_str}.csv')

        # combine vids into a set (drops duplicates): https://stackoverflow.com/a/45925961
        hourly_summary = data.groupby(['data_date', 'data_hour', 'rt', 'des']).agg({'vid': set, 'tatripid': set, 'tablockid': set}).reset_index()
        # get number of vehicles per hour per route
        hourly_summary['vh_count'] = hourly_summary['vid'].apply(len)
        hourly_summary['trip_count'] = hourly_summary['tatripid'].apply(len)
        hourly_summary['block_count'] = hourly_summary['tablockid'].apply(len)

        bucket.put_object(Body = hourly_summary.to_csv(index = False), 
                   Key = f'bus_hourly_summary_v2/{date_str}.csv')
        

Processing 2022-07-17 at 2022-08-08 21:52:03
------- loading data at 2022-08-08 21:52:03
------- parsing data at 2022-08-08 21:52:33
------- saving data at 2022-08-08 21:52:39
Processing 2022-07-18 at 2022-08-08 21:52:46
------- loading data at 2022-08-08 21:52:47
------- parsing data at 2022-08-08 21:53:20
------- saving data at 2022-08-08 21:53:29
Processing 2022-07-19 at 2022-08-08 21:53:42
------- loading data at 2022-08-08 21:53:42
------- parsing data at 2022-08-08 21:54:17
------- saving data at 2022-08-08 21:54:27
Processing 2022-07-20 at 2022-08-08 21:54:39
------- loading data at 2022-08-08 21:54:39
------- parsing data at 2022-08-08 21:55:13
------- saving data at 2022-08-08 21:55:22
Processing 2022-07-21 at 2022-08-08 21:55:35
------- loading data at 2022-08-08 21:55:35
------- parsing data at 2022-08-08 21:56:08
------- saving data at 2022-08-08 21:56:18
Processing 2022-07-22 at 2022-08-08 21:56:32
------- loading data at 2022-08-08 21:56:32
------- parsing data at 2022-08